In [ ]:
import sklearn
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
from sklearn.metrics import accuracy_score
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
from datetime import datetime
from sklearn.model_selection import RandomizedSearchCV, GridSearchCV
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from xgboost import XGBRegressor
from sklearn.preprocessing import LabelBinarizer
from xgboost import plot_importance
from matplotlib import pyplot
import matplotlib.pyplot as plt

In [ ]:
import hana_ml.dataframe as dataframe
from notebook_hana_connector.notebook_hana_connector import NotebookConnectionContext
conn = NotebookConnectionContext(connectionId = 'hanapoc')

In [ ]:
pd.set_option("display.max_rows", 50, "display.max_columns", None)

Get data

In [ ]:
sql = (
"select "
"top 10000 * "
"from TBL_MES_PRODUCTION_SUMMARY_EXT "
"where WORKCENTRE = 'RIGID04' and START_TIME > '2021'"
)
    
po_sel = conn.sql(sql)
pos = po_sel.collect()
pos

In [ ]:
expla = pos[['MAT_COUNT_PREV_DAY','MAT_COUNT_PREV_WEEK','ACTUAL_DURATION_CALC','NO_OF_CATS','TPM_COUNT_PREV_DAY','MAT_BACK_PNL','MAT_BLANKS','MAT_BOARD','MAT_CABINET','MAT_CAROUSEL','MAT_DRAWER_BX','MAT_DRWBOTTOM','MAT_EDGE_BAND','MAT_FASTENING','MAT_FITTINGS','MAT_GLUE','MAT_GO_FOBLNK','MAT_LEAFLETS','MAT_OBS','MAT_PACKAGING','MAT_PANELS','MAT_PHANTOMS','MAT_PLASTICS','MAT_RAILS','MAT_SELCO_BLK','MAT_SUSPN_BKT']]
dep = pos[['UNPLANNED_STOPS']]


One-hot-encode the MATERIAL column so that it can be used as input for modelling

In [ ]:
jobs_encoder = LabelBinarizer()
jobs_encoder.fit(pos['MATERIAL'])
transformed = jobs_encoder.transform(pos['MATERIAL'])
ohe_df = pd.DataFrame(transformed, columns=jobs_encoder.classes_)
data = pd.concat([expla, ohe_df], axis=1)

data.head(300)

Split into 70% train, 30% test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(data, dep, test_size=0.3, random_state=1) # 70% training and 30% test

Fit a XGBoost regression model

In [ ]:
model = XGBRegressor()
model.fit(X_train,y_train)

Calculate the R^2 score for the trained regression model

In [ ]:
# make predictions for test data
y_pred = model.predict(X_test)
predictions = [round(value) for value in y_pred]
# evaluate predictions
accuracy = r2_score(y_test, predictions)
print("R2_score: %.2f%%" % (accuracy * 100.0))

Plot feature importance for the trained model

In [ ]:
plot_importance(model, max_num_features=20)